#### POD NOTEBOOK

This notebook applies POD-coupled FFD to a geometry and illustrates various reconstruction aspects.

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import os
import random

from numpy import linalg as LA
from scipy.stats import qmc

from aero_optim.geom import split_profile
from aero_optim.ffd.ffd import FFD_2D

The notebook input variables are:

- `seed` the sampling seed 
- `ncontrol` the number of FFD control points on each side of the lattice box
- `bounds` the FFD deformation boundaries
- `file` the path to the file containing the geometry coordinates
- `nprofile` the number of FFD deformed profiles used to build the FFD dataset
- `nmode` the reduced dimension of the POD-coupled FFD

**Note**: the number of FFD design variables is `2 * ncontrol`, the number of POD-coupled design variables is `nmode`

In [ ]:
seed = 123
ncontrol = 4
bounds = (-0.2, 0.2)
file = "/path/to/aero-optim/examples/LRN-CASCADE/data/lrn_cascade.dat"
nprofile = 1000
nmode = 4

The `FFD_2D` object is created

In [ ]:
ffd = FFD_2D(file, ncontrol)

A random LHS sampler is built and used to sample the FFD dataset

**Note**: this should take about 10 seconds

In [ ]:
sampler = qmc.LatinHypercube(d=2 * ncontrol, seed=seed)
sample = sampler.random(n=nprofile)
scaled_sample = qmc.scale(sample, *bounds)

profiles = []
for Delta in scaled_sample:
    profiles.append(ffd.apply_ffd(Delta))

The POD eigenproblem is built and solved

In [ ]:
S = np.stack([p[:, -1] for p in profiles] , axis=1)
print(f"S shape: {S.shape}")
S_mean = 1 / nprofile * np.sum(S, axis=1)
print(f"S_mean shape: {S_mean.shape}")
F = S[:, :] - S_mean[:, None]
print(f"shape of F: {F.shape}")
C = np.matmul(np.transpose(F), F)
print(f"shape of C: {C.shape}")
eigenvalues, eigenvectors = LA.eigh(C)
print(f"shape of V: {eigenvectors.shape}")
phi = np.matmul(F, eigenvectors)
print(f"shape of phi: {phi.shape}")

The reduced matrices are computed based on `nmode`:

- `phi_tilde` the reduced eigenmode matrix
- `V_tilde_inv` the modal coefficients matrix
- `D_tilde` the reduced profiles matrix

In [ ]:
phi_tilde = phi[:, -nmode:]
print(f"shape of phi_tilde: {phi_tilde.shape}")
V_tilde_inv = np.linalg.inv(eigenvectors)[-nmode:, :]
print(f"shape of V_tilde_inv: {V_tilde_inv.shape}")
D_tilde = S_mean[:, None] + np.matmul(phi_tilde, V_tilde_inv)

Random profiles from the FFD dataset and their reduced reconstruction are plotted

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Times",
    "figure.dpi": 300,
    "font.size": 8,
    'legend.fontsize': 8, 
    "axes.titlesize": 8,
    "axes.labelsize": 8
})
fig, ax = plt.subplots(figsize=(5.2, 3.64))
for ii in range(5):
    idx = random.randint(0, len(profiles) - 1)
    ax.plot(profiles[idx][:, 0], S[:, idx])
    ax.plot(profiles[idx][:, 0], D_tilde[:, idx], linestyle="dashed", color="k")
ax.set(xlabel="$x$ [m]", ylabel="$y$ [m]", title=f"Reconstructed profiles with {nmode} modes")

The cumulative energy percentage is computed given `nmode`

In [ ]:
for nn in range(1, nmode + 1):
    print(f"{nn} mode energy percentage = {np.sum(eigenvalues[-nn:]) / sum(eigenvalues) * 100} %")

The geometric modes (a) and the energy and error (b) are plotted

In [ ]:
eigen_nrj = []
error = []
color = ["k", "r", "g", "b", "orange", "darkviolet"]
ls = ["solid", "dotted", "dashed", "dashdot", (0, (1, 1)), (0, (2, 1))]
fig = plt.figure(figsize=(6.5, 3.25))
fig.subplots_adjust(hspace=0.25)
ax1 = plt.subplot(1, 2, 1)  # geom. modes
ax2 = plt.subplot(1, 2, 2)  # POD eigenvalue energy
for nn in range(1, nmode + 1):
    # ax1
    ax1.plot(ffd.pts[:, 0] / 0.07, phi_tilde[:, -nn], label=f"mode {nn}", color=color[nn - 1], linestyle=ls[nn - 1])
for nn in range(1, 2 * ncontrol + 1):
    phi_tilde_tmp = phi[:, -nn:]
    V_tilde_inv_tmp = np.linalg.inv(eigenvectors)[-nn:, :]
    D_tilde_tmp = S_mean[:, None] + np.matmul(phi_tilde_tmp, V_tilde_inv_tmp)
    # ax2
    eigen_nrj.append(eigenvalues[-nn] / np.sum(eigenvalues) * 100)
    error.append(np.sqrt(np.sum([np.sum((y_true - y_pred)**2) for y_true, y_pred in zip(S.transpose(), D_tilde_tmp.transpose())]) / nprofile))
ax2.axvline(nmode, color="k", linestyle="dashed")
ax2.plot(range(1, len(eigen_nrj) + 1), eigen_nrj, color="blue", marker="s", ms=5, label="energy")
ax22 = ax2.twinx()  # instantiate a second Axes that shares the same x-axis
ax22.plot(range(1, len(error) + 1), error, color="red", marker="s", ms=5, label="RMSE")
ax22.set_yscale("log")
ax1.set(xlabel="$x / c$ [-]", ylabel="POD basis [-]", title="a) Geometric modes")
ax1.legend(loc="lower left")
ax2.set(xlabel="$N_i$ [-]", ylabel="$\\lambda_i / \\sum_{n=1}^{N_m} \\lambda_n$ [\%]", title="b) Energy and error")
ax22.set(ylabel="RMSE [m]")
lines, labels = ax2.get_legend_handles_labels()
lines2, labels2 = ax22.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc="center left", bbox_to_anchor=(0.5, 0.5))
plt.tight_layout()
plt.savefig(os.path.join(os.getcwd(), "POD.pdf"), bbox_inches="tight")
plt.show()

The POD boundaries are inferred from the modal coefficient min/max values

In [ ]:
l_bound = np.array([min(v) for v in V_tilde_inv])
u_bound = np.array([max(v) for v in V_tilde_inv])

In [ ]:
y_min = S_mean + np.sum(phi_tilde * np.array(l_bound), axis=1)
y_max = S_mean + np.sum(phi_tilde * np.array(u_bound), axis=1)

The non-correspondence between the POD and FFD boundaries is illustrated

In [ ]:
# FFD
min_profile = ffd.apply_ffd(np.array([bounds[0]] * 2 * ncontrol))
ffd_upper_min, ffd_lower_min = split_profile(min_profile)
max_profile = ffd.apply_ffd(np.array([bounds[-1]] * 2 * ncontrol))
ffd_upper_max, ffd_lower_max = split_profile(max_profile)
ffd_upper_pro, ffd_lower_pro = split_profile(ffd.pts)
# POD
upper_min, lower_min = split_profile(np.column_stack((ffd.pts[:, 0], y_min)))
upper_max, lower_max = split_profile(np.column_stack((ffd.pts[:, 0], y_max)))
upper_pro, lower_pro = split_profile(ffd.pts)
# Figure
fig = plt.figure(figsize=(6.5, 4))
fig.subplots_adjust(hspace=1.5, wspace=2)
gs = gridspec.GridSpec(2, 4, figure=fig)
ax1 = plt.subplot(gs[0, 1:3])
ax2 = plt.subplot(gs[1, :2])
ax3 = plt.subplot(gs[1, 2:])
# ax1
ax1.plot(ffd.pts[:, 0], ffd.pts[:, 1], label="baseline profile", color="k", linewidth=1)
ax1.plot(ffd.pts[:, 0], y_min, label=f"min profile", color="b", linewidth=1)
ax1.plot(ffd.pts[:, 0], y_max, label=f"max profile", color="r", linewidth=1)
ax1.fill_between(lower_min[:, 0], lower_min[:, 1], lower_max[:, 1], color="b", alpha=0.1)
ax1.fill_between(upper_max[:, 0], upper_min[:, 1], upper_max[:, 1], color="r", alpha=0.1)
ax1.plot(max_profile[:, 0], max_profile[:, 1], label=f"ffd min profile", color="b", linewidth=1, linestyle="dashed")
ax1.plot(min_profile[:, 0], min_profile[:, 1], label=f"ffd min profile", color="r", linewidth=1, linestyle="dashed")
# ax2
ax2.plot(ffd.pts[:, 0], ffd.pts[:, 1], label="baseline profile", color="k", linewidth=1)
ax2.plot(ffd.pts[:, 0], y_min, label=f"min profile", color="b", linewidth=1)
ax2.fill(ffd.pts[:, 0], y_min, label=f"min profile", color="b", alpha=0.1)
ax2.plot(max_profile[:, 0], max_profile[:, 1], label=f"ffd min profile", color="b", linewidth=1, linestyle="dashed")
# ax3
ax3.plot(ffd.pts[:, 0], ffd.pts[:, 1], label="baseline profile", color="k", linewidth=1)
ax3.plot(ffd.pts[:, 0], y_max, label=f"max profile", color="r", linewidth=1)
ax3.fill(ffd.pts[:, 0], y_max, label=f"max profile", color="r", alpha=0.1)
ax3.plot(min_profile[:, 0], min_profile[:, 1], label=f"ffd min profile", color="r", linewidth=1, linestyle="dashed")
# plot lattice grid
ax1.set(xlabel="$x$ [m]", ylabel="$y$ [m]", title="a) Design space")
ax2.set(xlabel="$x$ [m]", ylabel="$y$ [m]", title="b) Lower bound deformation")
ax3.set(xlabel="$x$ [m]", ylabel="$y$ [m]", title="c) Upper bound deformation")
# legend and display
plt.tight_layout()

The non-conservation of the FFD maximal deformations is illustrated 

In [ ]:
new_bounds = (l_bound, u_bound)
new_sampler = qmc.LatinHypercube(d=nmode, seed=seed)
new_sample = new_sampler.random(n=100)
scaled_sample = qmc.scale(new_sample, *new_bounds)
print(new_bounds)

In [ ]:
fig, ax = plt.subplots(figsize=(3.15, 2))
for ss_id, ss in enumerate(scaled_sample):
    y = S_mean + np.sum(phi_tilde * ss, axis=1)
    if ss_id == 0:
        ax.plot(ffd.pts[:, 0], y, linestyle="solid", color="lightgrey", linewidth=0.5, label=" random POD profiles")
    else:
        ax.plot(ffd.pts[:, 0], y, linestyle="solid", color="lightgrey", linewidth=0.5)
ax.plot(min_profile[:, 0], min_profile[:, 1], color="k", linewidth=1, linestyle="dashed", label="min/max FFD profiles")
ax.plot(max_profile[:, 0], max_profile[:, 1], color="k", linewidth=1, linestyle="dashed")
ax.legend()
ax.set(xlabel="$x$ [m]", ylabel="$y$ [m]")
plt.tight_layout()
plt.savefig(os.path.join(os.getcwd(), "POD_profiles.pdf"), bbox_inches="tight")
plt.show()

The latent space distributions are plotted

**Note**: if the number of modes `nmode` is changed, the sub-figure structure must be adapted

In [ ]:
latent_space = V_tilde_inv

In [ ]:
fig = plt.figure(figsize=(6.5, 6.5))
fig.subplots_adjust(hspace=0.25, wspace=0.25)
ax1 = plt.subplot(2, 2, 1)  # dim 1
ax2 = plt.subplot(2, 2, 2)  # dim 2
ax3 = plt.subplot(2, 2, 3)  # dim 3
ax4 = plt.subplot(2, 2, 4)  # dim 4
ax1.hist(latent_space[-1, :], bins=20, linewidth=0.5, edgecolor="white")
ax1.set(xlabel="$\\alpha_1$ [-]", ylabel="$N$ [-]", title="a) distribution /dim 1")
ax2.hist(latent_space[-2, :], bins=20, linewidth=0.5, edgecolor="white")
ax2.set(xlabel="$\\alpha_2$ [-]", ylabel="$N$ [-]", title="b) distribution /dim 2")
ax3.hist(latent_space[-3, :], bins=20, linewidth=0.5, edgecolor="white")
ax3.set(xlabel="$\\alpha_3$ [-]", ylabel="$N$ [-]", title="c) distribution /dim 3")
ax4.hist(latent_space[-4, :], bins=20, linewidth=0.5, edgecolor="white")
ax4.set(xlabel="$\\alpha_4$ [-]", ylabel="$N$ [-]", title="d) distribution /dim 4")